# Lab 3: Prompt Tuning - Deployment Guide
---
## 🎯 實驗目標

本 Notebook 展示 Prompt Tuning 的核心特性:**軟提示的獨立部署與多任務切換**。

與其他 PEFT 方法不同,Prompt Tuning 的軟提示參數**無需合併**到基礎模型中,這帶來獨特的部署優勢:

### 關鍵學習要點
- 理解 Prompt Tuning 的部署特性與其他 PEFT 方法的差異
- 掌握軟提示的儲存、載入與版本管理
- 實現多任務軟提示的動態切換機制
- 優化生產環境的推理效能
- 學習最佳實踐與故障排除策略

---

## 1. 環境設置與依賴檢查

In [ ]:
# 導入必要的函式庫
import torch
import time
import gc
import os
import json
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline
)
from peft import (
    PeftModel,
    PromptTuningConfig,
    get_peft_model,
    TaskType
)
from datasets import load_dataset

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用設備: {device}")

# 設定隨機種子
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

## 2. Prompt Tuning 部署特性分析

### 2.1 為什麼 Prompt Tuning 不需要合併?

**核心原理**:
```
Prompt Tuning: [軟提示向量] + [輸入文本] → 模型處理
LoRA/IA³:      修改權重矩陣 W' = W + ΔW → 可合併
```

**關鍵差異對比**:

| 對比維度 | Prompt Tuning | LoRA | IA³ |
|---------|--------------|------|-----|
| **參數位置** | 輸入層(軟提示嵌入) | 權重矩陣分解 | 縮放向量 |
| **是否可合併** | ❌ 不需要 | ✅ 可合併 | ✅ 可合併 |
| **部署方式** | 軟提示 + 基礎模型 | 合併後單一模型 | 合併後單一模型 |
| **推理開銷** | 軟提示長度影響 | 零開銷(合併後) | 零開銷(合併後) |
| **多任務切換** | **極其便捷** | 需重新載入 | 需重新載入 |
| **儲存成本** | 極低(僅軟提示) | 中等 | 低 |

### 2.2 Prompt Tuning 的獨特部署優勢

1. **多任務共享基礎模型**: 一個模型 + N 個軟提示 = N 個任務能力
2. **動態任務切換**: 僅需替換軟提示,無需重新載入模型
3. **版本管理簡單**: 軟提示參數量極小,易於版本控制與實驗追蹤
4. **記憶體效率高**: 多任務部署僅增加少量記憶體開銷

## 3. 載入訓練好的 Prompt Tuning 模型

In [ ]:
# 模型與適配器路徑
base_model_name = "t5-small"
adapter_path = "./t5-prompt-tuning-billsum"  # 假設這是訓練好的適配器路徑

# 載入基礎模型與分詞器
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float32,  # T5 通常使用 FP32
    device_map="auto" if torch.cuda.is_available() else None
)

print(f"基礎模型參數量: {base_model.num_parameters():,}")
print(f"基礎模型記憶體占用: {base_model.get_memory_footprint() / 1024**2:.2f} MB")

In [ ]:
# 載入 Prompt Tuning 適配器
if os.path.exists(adapter_path):
    # 從檢查點載入最新的適配器
    checkpoints = [d for d in os.listdir(adapter_path) if d.startswith("checkpoint-")]
    if checkpoints:
        latest_checkpoint = max(
            [os.path.join(adapter_path, d) for d in checkpoints],
            key=os.path.getmtime
        )
        print(f"載入最新檢查點: {latest_checkpoint}")
        peft_model = PeftModel.from_pretrained(base_model, latest_checkpoint)
    else:
        # 直接從目錄載入
        peft_model = PeftModel.from_pretrained(base_model, adapter_path)
    
    print(f"✅ 成功載入 Prompt Tuning 適配器: {adapter_path}")
else:
    # 若無訓練好的適配器,創建示範配置
    print("未找到訓練好的適配器,創建示範 Prompt Tuning 配置...")
    
    prompt_config = PromptTuningConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        prompt_tuning_init="TEXT",
        prompt_tuning_init_text="Summarize the following text:",
        num_virtual_tokens=8,
        tokenizer_name_or_path=base_model_name
    )
    
    peft_model = get_peft_model(base_model, prompt_config)
    print("已創建示範 Prompt Tuning 模型")

# 顯示可訓練參數統計
peft_model.print_trainable_parameters()

## 4. 軟提示結構分析

In [ ]:
def analyze_soft_prompts(model):
    """分析軟提示結構與參數"""
    prompt_params = {}
    total_prompt_params = 0
    
    print("=== 軟提示參數分析 ===")
    for name, param in model.named_parameters():
        if 'prompt' in name.lower():
            prompt_params[name] = {
                'shape': param.shape,
                'dtype': param.dtype,
                'numel': param.numel()
            }
            total_prompt_params += param.numel()
            
            print(f"\n軟提示參數: {name}")
            print(f"  形狀: {param.shape}")
            print(f"  數據類型: {param.dtype}")
            print(f"  參數量: {param.numel():,}")
    
    total_params = sum(p.numel() for p in model.parameters())
    prompt_ratio = (total_prompt_params / total_params) * 100
    
    print(f"\n{'='*50}")
    print(f"軟提示總參數量: {total_prompt_params:,}")
    print(f"模型總參數量: {total_params:,}")
    print(f"軟提示占比: {prompt_ratio:.4f}%")
    print(f"{'='*50}")
    
    return prompt_params

soft_prompt_structure = analyze_soft_prompts(peft_model)

## 5. 推理效能基準測試

In [ ]:
def benchmark_summarization(model, tokenizer, test_samples, num_runs=3):
    """摘要生成效能基準測試"""
    model.eval()
    total_time = 0
    results = []
    
    with torch.no_grad():
        for run in range(num_runs):
            start_time = time.time()
            
            for sample in test_samples:
                # 準備輸入
                prompt = "summarize: " + sample['text'][:1024]  # 限制輸入長度
                inputs = tokenizer(
                    prompt,
                    return_tensors="pt",
                    max_length=1024,
                    truncation=True
                )
                
                if torch.cuda.is_available():
                    inputs = {k: v.cuda() for k, v in inputs.items()}
                
                # 生成摘要
                outputs = model.generate(
                    input_ids=inputs["input_ids"],
                    max_length=150,
                    num_beams=4,
                    early_stopping=True
                )
                
                generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
                
                if run == 0:  # 僅在第一輪保存結果
                    results.append({
                        'original': sample['text'][:200] + '...',
                        'reference': sample['summary'],
                        'generated': generated_summary
                    })
            
            end_time = time.time()
            total_time += (end_time - start_time)
    
    avg_time = total_time / num_runs
    return avg_time, results

# 載入測試樣本
print("載入測試數據集...")
test_dataset = load_dataset("billsum", split="test[:5]")

print("\n=== 推理效能測試 ===")
inference_time, inference_results = benchmark_summarization(
    peft_model,
    tokenizer,
    test_dataset
)

print(f"平均推理時間: {inference_time:.4f} 秒 (5 個樣本)")
print(f"每個樣本平均時間: {inference_time / len(test_dataset):.4f} 秒")
print(f"記憶體占用: {peft_model.get_memory_footprint() / 1024**2:.2f} MB")

In [ ]:
# 顯示生成範例
print("\n=== 摘要生成範例 ===")
for i, result in enumerate(inference_results[:2]):  # 僅顯示前兩個範例
    print(f"\n{'='*80}")
    print(f"範例 {i+1}")
    print(f"{'-'*80}")
    print(f"原始文本: {result['original']}")
    print(f"\n參考摘要: {result['reference']}")
    print(f"\n生成摘要: {result['generated']}")
    print(f"{'='*80}")

## 6. 軟提示儲存與版本管理

### 6.1 軟提示的獨立儲存

In [ ]:
# 設定儲存路徑
save_path = "./prompt-tuning-deployed"
os.makedirs(save_path, exist_ok=True)

print(f"=== 儲存 Prompt Tuning 軟提示到: {save_path} ===")

# 儲存軟提示適配器(不包含基礎模型)
peft_model.save_pretrained(save_path)

print("✅ 軟提示儲存完成!")

# 檢查儲存的檔案
saved_files = os.listdir(save_path)
print(f"\n儲存的檔案: {saved_files}")

# 計算軟提示檔案大小
total_size = 0
for file in saved_files:
    file_path = os.path.join(save_path, file)
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path)
        total_size += size
        print(f"  {file}: {size / 1024:.2f} KB")

print(f"\n軟提示總大小: {total_size / 1024:.2f} KB")
print(f"基礎模型大小(參考): ~242 MB (t5-small)")
print(f"儲存效率: {(total_size / (242 * 1024 * 1024)) * 100:.4f}% 的基礎模型大小")

### 6.2 軟提示配置分析

In [ ]:
# 讀取並顯示軟提示配置
config_path = os.path.join(save_path, "adapter_config.json")

if os.path.exists(config_path):
    with open(config_path, 'r', encoding='utf-8') as f:
        prompt_config = json.load(f)
    
    print("=== Prompt Tuning 配置詳情 ===")
    print(json.dumps(prompt_config, indent=2, ensure_ascii=False))
    
    print(f"\n關鍵配置參數:")
    print(f"  任務類型: {prompt_config.get('task_type', 'N/A')}")
    print(f"  軟提示長度: {prompt_config.get('num_virtual_tokens', 'N/A')} tokens")
    print(f"  初始化方法: {prompt_config.get('prompt_tuning_init', 'N/A')}")
    print(f"  初始化文本: {prompt_config.get('prompt_tuning_init_text', 'N/A')}")

## 7. 多任務軟提示管理與動態切換

### 7.1 模擬多任務場景

In [ ]:
class MultiTaskPromptManager:
    """
    多任務軟提示管理器
    
    實現功能:
    - 管理多個任務的軟提示
    - 動態切換任務
    - 記憶體效率優化
    """
    
    def __init__(self, base_model, tokenizer):
        self.base_model = base_model
        self.tokenizer = tokenizer
        self.task_prompts = {}  # 儲存任務特定的軟提示路徑
        self.current_task = None
        self.current_model = None
    
    def register_task(self, task_name, adapter_path):
        """註冊新任務的軟提示"""
        if os.path.exists(adapter_path):
            self.task_prompts[task_name] = adapter_path
            print(f"✅ 已註冊任務 '{task_name}': {adapter_path}")
        else:
            print(f"❌ 任務路徑不存在: {adapter_path}")
    
    def switch_task(self, task_name):
        """切換到指定任務"""
        if task_name not in self.task_prompts:
            raise ValueError(f"任務 '{task_name}' 尚未註冊")
        
        # 清理當前模型(若存在)
        if self.current_model is not None:
            del self.current_model
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # 載入新任務的軟提示
        adapter_path = self.task_prompts[task_name]
        self.current_model = PeftModel.from_pretrained(
            self.base_model,
            adapter_path
        )
        self.current_task = task_name
        
        print(f"🔄 已切換到任務: {task_name}")
    
    def generate(self, input_text, **generation_kwargs):
        """使用當前任務的軟提示生成輸出"""
        if self.current_model is None:
            raise RuntimeError("尚未選擇任務,請先調用 switch_task()")
        
        inputs = self.tokenizer(
            input_text,
            return_tensors="pt",
            max_length=1024,
            truncation=True
        )
        
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.current_model.generate(
                input_ids=inputs["input_ids"],
                **generation_kwargs
            )
        
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    def list_tasks(self):
        """列出所有已註冊的任務"""
        print("\n=== 已註冊的任務 ===")
        for task_name, adapter_path in self.task_prompts.items():
            status = "✓" if task_name == self.current_task else " "
            print(f"[{status}] {task_name}: {adapter_path}")

# 創建多任務管理器示範
manager = MultiTaskPromptManager(base_model, tokenizer)

# 註冊任務(實際使用時,這些應該是不同任務的軟提示)
manager.register_task("summarization", save_path)
# manager.register_task("translation", "./prompt-translation")
# manager.register_task("qa", "./prompt-qa")

manager.list_tasks()

### 7.2 測試任務切換

In [ ]:
# 切換到摘要任務
manager.switch_task("summarization")

# 測試生成
test_text = "summarize: " + test_dataset[0]['text'][:500]
generated = manager.generate(
    test_text,
    max_length=150,
    num_beams=4,
    early_stopping=True
)

print("\n=== 任務切換測試 ===")
print(f"當前任務: {manager.current_task}")
print(f"\n輸入文本: {test_dataset[0]['text'][:200]}...")
print(f"\n生成摘要: {generated}")

## 8. 生產部署配置

### 8.1 部署配置檔案

In [ ]:
# 創建部署配置檔案
deployment_config = {
    "model_info": {
        "base_model": base_model_name,
        "peft_method": "Prompt Tuning",
        "task_type": "Seq2Seq Summarization",
        "mergeable": False,
        "soft_prompt_tokens": prompt_config.get('num_virtual_tokens', 8),
        "soft_prompt_size_kb": total_size / 1024
    },
    "performance_metrics": {
        "avg_inference_time_seconds": inference_time / len(test_dataset),
        "memory_usage_mb": peft_model.get_memory_footprint() / 1024**2,
        "soft_prompt_overhead": "Minimal (input length reduction)"
    },
    "deployment_strategy": {
        "type": "Soft Prompt + Base Model",
        "base_model_loading": "Load once, reuse for multiple tasks",
        "task_switching": "Dynamic soft prompt swapping",
        "multi_task_support": True
    },
    "deployment_requirements": {
        "python_packages": [
            "torch>=1.9.0",
            "transformers>=4.20.0",
            "peft>=0.3.0"
        ],
        "minimum_gpu_memory_gb": 4,
        "recommended_gpu_memory_gb": 8
    },
    "usage_example": {
        "load_base_model": f"AutoModelForSeq2SeqLM.from_pretrained('{base_model_name}')",
        "load_soft_prompt": f"PeftModel.from_pretrained(base_model, '{save_path}')",
        "inference_note": "Add task-specific prefix (e.g., 'summarize:') to input"
    },
    "best_practices": {
        "multi_task_deployment": "Load base model once, swap soft prompts for different tasks",
        "version_control": "Store soft prompts separately for easy A/B testing",
        "scaling": "Base model can be shared across multiple task endpoints",
        "monitoring": "Track soft prompt switching frequency and cache hit rate"
    }
}

# 儲存配置檔案
config_output_path = os.path.join(save_path, "deployment_config.json")
with open(config_output_path, 'w', encoding='utf-8') as f:
    json.dump(deployment_config, f, indent=2, ensure_ascii=False)

print("=== 生產部署配置 ===")
print(json.dumps(deployment_config, indent=2, ensure_ascii=False))
print(f"\n✅ 配置檔案已儲存到: {config_output_path}")

### 8.2 部署架構圖

```
生產環境部署架構 (多任務場景)
═════════════════════════════════════════════════════════════

┌─────────────────────────────────────────────────────────┐
│                    API Gateway                          │
└────────────────┬────────────────────────────────────────┘
                 │
        ┌────────┼────────┐
        │        │        │
    ┌───▼──┐ ┌──▼───┐ ┌─▼────┐
    │Task A│ │Task B│ │Task C│
    │ (摘要)│ │(翻譯)│ │ (QA) │
    └───┬──┘ └──┬───┘ └─┬────┘
        │        │       │
        └────────┼───────┘
                 │
    ┌────────────▼──────────────┐
    │  Soft Prompt Selector     │  ← 動態切換軟提示
    ├───────────────────────────┤
    │  Prompt A  │ Prompt B │ C │  (僅幾 KB 大小)
    └────────────┬──────────────┘
                 │
    ┌────────────▼──────────────┐
    │    T5 Base Model          │  ← 共享基礎模型
    │    (僅載入一次)            │  (242 MB)
    └───────────────────────────┘

優勢:
✅ 單一基礎模型支援多任務
✅ 軟提示切換成本極低 (<1ms)
✅ 總記憶體占用 ≈ 1個基礎模型 + N個軟提示
✅ 易於 A/B 測試與版本迭代
```

## 9. Prompt Tuning 部署最佳實踐

### 9.1 何時選擇 Prompt Tuning 部署策略

In [ ]:
print("=== Prompt Tuning 部署場景推薦 ===")
print("")
print("🎯 最適合場景:")
print("  • 需要支援多個相似任務(共享基礎模型)")
print("  • 頻繁的模型版本迭代與 A/B 測試")
print("  • 使用超大規模模型(>10B 參數,規模效應顯著)")
print("  • 需要快速部署新任務能力")
print("  • 儲存與傳輸成本敏感(軟提示僅幾 KB)")
print("")
print("⚠️ 需要謹慎考慮的場景:")
print("  • 單一任務且對推理延遲極其敏感(軟提示會略微增加輸入長度)")
print("  • 小型模型(<1B 參數,規模效應不明顯)")
print("  • 任務需要大幅修改模型內部表示")
print("  • 需要極致的推理效能優化(考慮 LoRA/IA³ 合併)")
print("")
print("🔄 與其他方法的組合策略:")
print("  • Prompt Tuning + LoRA: 大模型用 Prompt Tuning,小模型用 LoRA")
print("  • 先用 Prompt Tuning 快速驗證,再用其他方法精調")
print("  • 不同層使用不同 PEFT 方法(混合策略)")

### 9.2 部署檢核清單

In [ ]:
print("=== Prompt Tuning 部署檢核清單 ===")
print("")
print("📋 部署前檢查:")
print("  □ 確認軟提示訓練完成並儲存")
print("  □ 在驗證集上驗證軟提示效果")
print("  □ 記錄軟提示配置(長度、初始化方法等)")
print("  □ 測試基礎模型 + 軟提示的載入流程")
print("")
print("🔄 多任務部署:")
print("  □ 設計軟提示命名與版本管理規範")
print("  □ 實現軟提示動態載入與切換機制")
print("  □ 配置軟提示快取策略(熱門任務常駐記憶體)")
print("  □ 監控任務切換頻率與效能影響")
print("")
print("✅ 部署後驗證:")
print("  □ 推理速度測試")
print("  □ 記憶體占用監控")
print("  □ 輸出品質抽樣檢查")
print("  □ 長時間穩定性測試")
print("  □ 軟提示切換正確性驗證(多任務場景)")
print("")
print("📚 文件與監控:")
print("  □ 儲存部署配置檔案")
print("  □ 記錄效能基準數據")
print("  □ 準備回滾方案(保留多版本軟提示)")
print("  □ 更新 API 文件與使用範例")
print("  □ 設置軟提示版本追蹤系統")

### 9.3 故障排除指南

In [ ]:
print("=== Prompt Tuning 常見問題與解決方案 ===")
print("")
print("❓ 問題 1: 載入軟提示時出現維度不匹配")
print("   原因: 軟提示訓練時的基礎模型版本與部署時不一致")
print("   解決: 確保使用完全相同的基礎模型(名稱與版本)")
print("")
print("❓ 問題 2: 推理結果與訓練時差異很大")
print("   原因: 可能缺少任務前綴(如 'summarize:')")
print("   解決: 檢查輸入格式,確保包含正確的任務前綴")
print("")
print("❓ 問題 3: 多任務切換後輸出異常")
print("   原因: 軟提示未正確切換或基礎模型狀態污染")
print("   解決: 確保每次切換都完整載入新軟提示,清空 CUDA cache")
print("")
print("❓ 問題 4: 記憶體占用持續增長")
print("   原因: 多次軟提示切換未釋放舊模型")
print("   解決: 在切換前執行 del model + gc.collect() + torch.cuda.empty_cache()")
print("")
print("❓ 問題 5: 生成品質低於預期")
print("   原因: 可能使用了小型基礎模型(規模效應不足)")
print("   解決: 考慮升級到更大的基礎模型(>1B),或改用 LoRA 方法")

## 10. PEFT 方法部署策略對比總結

In [ ]:
# PEFT 方法部署策略對比
deployment_comparison = {
    "Prompt Tuning": {
        "can_merge": False,
        "deployment_unit": "Soft Prompt (幾 KB) + Base Model",
        "inference_overhead": "Input length increase",
        "multi_task_support": "Excellent (dynamic switching)",
        "deployment_complexity": "Medium",
        "best_for": "Multi-task, large models, frequent updates"
    },
    "LoRA": {
        "can_merge": True,
        "deployment_unit": "Merged single model OR Adapter + Base Model",
        "inference_overhead": "Zero (if merged)",
        "multi_task_support": "Good (need reload)",
        "deployment_complexity": "Simple (if merged)",
        "best_for": "Single task, balanced efficiency and quality"
    },
    "IA3": {
        "can_merge": True,
        "deployment_unit": "Merged single model",
        "inference_overhead": "Zero (fully merged)",
        "multi_task_support": "Medium (need reload)",
        "deployment_complexity": "Simple",
        "best_for": "Extreme efficiency, inference-critical"
    },
    "Prefix Tuning": {
        "can_merge": False,
        "deployment_unit": "Prefix + Base Model",
        "inference_overhead": "Attention computation increase",
        "multi_task_support": "Good (dynamic switching)",
        "deployment_complexity": "Complex",
        "best_for": "Generation tasks, need expressiveness"
    },
    "AdapterLayers": {
        "can_merge": False,
        "deployment_unit": "Adapter Layers + Base Model",
        "inference_overhead": "Bottleneck layer computation",
        "multi_task_support": "Excellent (modular design)",
        "deployment_complexity": "Medium",
        "best_for": "Multi-task modular deployment"
    }
}

print("=== PEFT 方法部署策略對比 ===")
print(f"{'方法':<15} {'可合併':<10} {'部署單元':<30} {'推理開銷':<20} {'多任務支援':<20}")
print("-" * 100)

for method, info in deployment_comparison.items():
    mergeable = "✅" if info["can_merge"] else "❌"
    print(f"{method:<15} {mergeable:<10} {info['deployment_unit']:<30} {info['inference_overhead']:<20} {info['multi_task_support']:<20}")

print("\n=== Prompt Tuning 獨特部署優勢 ===")
print("✅ 多任務共享: 一個基礎模型 + N 個軟提示 = N 個任務能力")
print("✅ 動態切換: 軟提示切換成本極低(<1ms)")
print("✅ 版本管理: 軟提示檔案極小,易於版本控制與實驗追蹤")
print("✅ 規模友好: 大模型上效果逼近全參數微調,但儲存成本極低")
print("✅ 無需合併: 保持基礎模型原始狀態,易於回滾與更新")

## 11. 實驗總結

### 11.1 核心學習成果

通過本實驗,我們深入探索了 **Prompt Tuning 的獨特部署特性**:

1. **理論理解**: 掌握了為何 Prompt Tuning 不需要也不應該合併的原理
2. **實踐經驗**: 完成了軟提示的儲存、載入、版本管理全流程
3. **多任務管理**: 實現了基於軟提示動態切換的多任務部署架構
4. **最佳實踐**: 學習了生產環境部署的完整檢核清單與故障排除策略

### 11.2 Prompt Tuning 的獨特價值

- **多任務友好**: 單一基礎模型支援無限任務,僅需切換軟提示
- **極致儲存效率**: 每個任務僅需幾 KB 的軟提示檔案
- **部署靈活性**: 無需修改基礎模型,易於版本管理與回滾
- **規模驅動**: 隨著模型規模增大,效果逐步逼近全參數微調

### 11.3 部署策略選擇建議

| 場景 | 推薦方法 | 理由 |
|------|---------|------|
| **多任務系統** | **Prompt Tuning** | 共享基礎模型,動態切換 |
| **單任務高效能** | LoRA(合併) 或 IA³ | 零推理開銷 |
| **頻繁更新迭代** | Prompt Tuning | 版本管理簡單 |
| **極致推理速度** | IA³(合併) | 完全無開銷 |
| **大模型(>10B)** | Prompt Tuning | 規模效應顯著 |
| **小模型(<1B)** | LoRA | 表達能力更強 |

Prompt Tuning 完美詮釋了 **「簡潔而不簡單」** 的設計哲學,在適當場景下,極簡的方法往往帶來最優雅的解決方案。

## 12. 資源清理

In [ ]:
# 清理實驗資源
print("=== 清理實驗資源 ===")

# 清理模型
if 'peft_model' in locals():
    del peft_model
if 'base_model' in locals():
    del base_model
if 'manager' in locals():
    del manager

# 垃圾回收
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("✅ 資源清理完成")
print("\n🎉 Lab 3 - Prompt Tuning 部署指南實驗完成!")
print(f"軟提示已儲存到: {save_path}")
print("您現在可以將軟提示與基礎模型分別部署,享受多任務動態切換的便利!")